In [ ]:
# !pip install faiss
# !pip install faiss-cpu
# !pip install sentence-transformers

In [1]:
import pandas as pd

pd.set_option("display.max_colwidth", 100)

In [2]:
df = pd.read_csv("sample_text.csv")
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [3]:
df.shape

(8, 2)

In [4]:
from sentence_transformers import SentenceTransformer

#encoder = SentenceTransformer("paraphrase-MiniLM-L6-v2")
#vectors = encoder.encode(df['text'])
#vectors.shape

In [5]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df['text'])
vectors.shape

# From huggingface we are using a model or a transformer entitie. in simple lang they are coverting the text into vectors
# encoder.encode expects an array of text and that is stored in vector


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\KRISHNA\anaconda3\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\KRISHNA\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(8, 768)

In [6]:
vectors

array([[-0.00247396,  0.03626723, -0.05290459, ..., -0.09152357,
        -0.03970001, -0.04330489],
       [-0.03357268,  0.00980517, -0.03250129, ..., -0.05165466,
         0.02245888, -0.03156181],
       [-0.01865322, -0.04051315, -0.01235386, ...,  0.00610586,
        -0.07179644,  0.02773852],
       ...,
       [-0.00066459,  0.04252128, -0.05645508, ...,  0.01315471,
        -0.03183568, -0.04357662],
       [-0.03317154,  0.03252454, -0.02484838, ...,  0.0117442 ,
         0.05747125,  0.00571021],
       [-0.00166395,  0.00413827, -0.04597083, ...,  0.02008528,
         0.05656242, -0.00161596]], dtype=float32)

In [7]:
dim= vectors.shape[1]
dim

# Save the size of vector so we can use in faiss V_DB

768

In [8]:
import faiss

index = faiss.IndexFlatL2(dim)
index

# IndexFlatL2 the index that use Euclidean distance or L2 distance

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000002242A9DD390> >

In [9]:
index.add(vectors)

In [10]:
search_query = "I want to buy a polo t-shirt"

vec = encoder.encode(search_query)
vec.shape

(768,)

In [11]:
import numpy as np

svec = np.array(vec).reshape(1, -1)
svec.shape

(1, 768)

In [12]:
index.search(svec, k = 2)

# this search vector expect 2d array
# k is like similar to knearest neighbour , tell how many similar vector you want

(array([[1.3844835, 1.4039094]], dtype=float32), array([[3, 2]], dtype=int64))

- The first one is distance and second one is index in our original dataframe 
- 3,2 both represent the fashion, and the query we gave is also similar to fashion 

In [13]:
distance, I = index.search(svec, k = 2)

In [14]:
I

array([[3, 2]], dtype=int64)

In [15]:
distance

array([[1.3844835, 1.4039094]], dtype=float32)

In [16]:
search_query

'I want to buy a polo t-shirt'

- In this search_query the exact word is not in the 3,2. It's not like a key word search ..
- it's a semantic search 
    - It is capturing the meaning of the context and giving giving the similar sentence here

In [19]:
df.loc[[3,2]]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion
